# Práctica 1. Diferenciación de entidades.
## Autores: 
     Raúl Sánchez Martín
     Ignacio Arias Barra

In [259]:
import re, pprint, os, numpy
# -*- coding: utf-8 -*-
import nltk
import string
#path_to_append = '/media/nacho/f8371289-0f00-4406-89db-d575f3cdb35e/Master/Trimestre 2/RIM/nltk_data'
#nltk.data.path.append(path_to_append)
from sklearn.metrics.cluster import *
from sklearn.cluster import AgglomerativeClustering
from nltk.cluster import GAAClusterer
from sklearn.metrics.cluster import adjusted_rand_score
from nltk.corpus import stopwords
import operator


## Funciones

### Código origen
Lectura de fichero:

In [260]:
def read_file(file):
    myfile = open(file,"r")
    data = ""
    lines = myfile.readlines()
    for line in lines:
        data = data + line
    myfile.close
    return data


Clustering:

In [261]:
def cluster_texts(texts, clustersNumber, distance):
    # Convierte textos en una coleccion
    # Load the list of texts into a TextCollection object.
    collection = nltk.TextCollection(texts)    
#     print("Created a collection of", len(collection), "terms.")
    
    # Get a list of unique terms
    unique_terms = list(set(collection))
#     print("Unique terms found: ", len(unique_terms))

    ### And here we actually call the function and create our array of vectors.
    # TF mide la frecuencia en los textos.
    # Mira de los terminos unicos, cuantas veces aparece en el documento. No mira cuantas veces aparece en la coleccion
    # Hay otras medidas, como TF-IDF que son mas precisas porque tambien miran cuantas veces aparece en la coleccion
    vectors = [numpy.array(TF(f,unique_terms, collection)) for f in texts]
    # print("Vectors created.")
    # print(vectors)
    
#     for vector in vectors:
#         print("Vector ", len(vector))

#     # initialize the clusterer
#     clusterer = GAAClusterer(clustersNumber)
#     clusters = clusterer.cluster(vectors, True)
    # Estas lineas siguientes comentadas es lo mismo pero con otra libreria, la llamada scikit-learn
    clusterer = AgglomerativeClustering(n_clusters=clustersNumber,
                                     linkage="average", affinity=distanceFunction)
    clusters = clusterer.fit_predict(vectors)

    return clusters

Frecuencia de términos únicos en un documento

In [262]:
# Function to create a TF vector for one document. For each of
# our unique words, we have a feature which is the tf for that word
# in the current document
def TF(document, unique_terms, collection):
    word_tf = []
    for word in unique_terms:
        word_tf.append(collection.tf(word, document))
    return word_tf

### Nuevo código

*FUNCIÓN PARA DETECTAR EL IDIOMA DE UN TEXTO*

In [263]:
def get_language(possible_lan, text):
    # More info in: http://blog.alejandronolla.com/2013/05/15/detecting-text-language-with-python-and-nltk/
    languages_score = {}
    for language in possible_lan:
        stopwords_set = set(stopwords.words(language))
        words_set = set(text)
        common_elements = words_set.intersection(stopwords_set)
        languages_score[possible_lan[language]] = len(common_elements)
    return max(languages_score.items(), key=operator.itemgetter(1))[0]
        


*EVALUACIÓN DIFERENTES TAMAÑOS DE CLUSTER*

In [264]:
def eval_test(top_test, priori_groups, reference):
    score = 0
    curr_score = -9999999
    curr_priori_groups = 1
    for n in range(top_test):
        test = cluster_texts(texts,priori_groups,distanceFunction)
        # print("test: ", test)
        # Evaluation
        score = adjusted_rand_score(reference, test)

        print("group: ", priori_groups)
        print("rand_score: ", score)

        if score >= curr_score:
            curr_score = score
            curr_priori_groups = priori_groups

        priori_groups+=1

    # Gold Standard
    # el documento que meto el primero sera igual que los que esten en todos los ceros
    print("\nBest agroupation: ", curr_priori_groups)
    print("Best score: ", curr_score)
    print("reference: ", reference)
    return curr_priori_groups

*USO DE ENTIDADES I*

Se hará uso de las entidades reconocidas utilizando el método *ne_chunk* de la propia librería *nltk*. Simplemente se diferenciará entre nombres, adjetivos, adverbios, etc...

In [265]:
def get_named_entities_1(initial_document, selected_types):
    named_entities = list()
    selected_entities = list()
    try:
        for sentence in initial_document:
            tokenized_sentence = nltk.word_tokenize(sentence)
            # tagged_sentence = nltk.pos_tag(tokenized_sentence)
            tagged_sentence = nltk.pos_tag(tokenized_sentence, tagset= 'universal')
            named_ent = nltk.ne_chunk(tagged_sentence, binary=False)
            named_entities.append(named_ent)

        for element in named_entities:
            word = element.pos()[0][0][0]
            type_word = element.pos()[0][0][1]
            if type_word in selected_types:
                selected_entities.append(word)
        
        return selected_entities
    
    except Exception as e:
        print(str(e))

        
def get_named_ent_txts_1(raw_texts):
    named_ent_txts_1 = []
    for text in raw_texts:
        curr_named_ent = get_named_entities_1(text, types_included_1)
        text = nltk.Text(curr_named_ent)
        named_ent_txts_1.append(curr_named_ent)
    return named_ent_txts_1

*ELIMINAR PALABRAS VACÍAS*

In [266]:
from nltk.corpus import stopwords


# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# # with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()


def delete_stopwords(text,lan='en'):

    if lan == 'en':        
        # Obtenemos las sentencias del texto.
        sentences = nltk.sent_tokenize(text)
        # Obtenemos las "stopwords" del inglés.
        stop = set(stopwords.words('english'))

    elif lan == 'es':        
        # Elegimos el tokenizador para el español de la NLTK.
        spanish_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
        # Obtenemos las sentencias del texto.
        sentences = spanish_tokenizer.tokenize(text)        

        # Obtenemos las "stopwords" del español.
        stop = set(stopwords.words('spanish'))
      
    print("Frases originales: ")
    print(sentences)
    
    stopw = []
    for i in stop:
        stopw.append(i)
    print("Las stop words en lan = " + lan)
    print(stopw)
    print()

    # Eliminamos las palabras que coinciden con alguna "stopword".
    print("Frases procesadas sin stopwords: ")
    for sentence in sentences:
        non_stop_sentence = ""
        for word in sentence.lower().split():
            if word not in stop:
                non_stop_sentence = non_stop_sentence + word + " "
        # Imprimimos las sentencias procesadas.
        print(non_stop_sentence)
    return non_stop_sentence

# espanol = delete_stopwords(text,lan='es')
# ingles = delete_stopwords(text)

*ELIMINAR EL NOMBRE DE LA PERSONA EN LOS TEXTOS*

*REPRESENTACIÓN DE NGRAMAS A NIVEL DE PALABRAS*

(trigramas es poner casa-perro-coche en las ocurrencias por texto),
 se puede hacer los ngramas con nltk y scikit-learn

*STEAMMING*

In [267]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()

def stemming_original(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
    
    # Obtenemos los tokens del texto.
    tokens = nltk.word_tokenize(text)
        
    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    print("Palabras originales: ")
    print(tokens)
    print("Palabras procesadas con Snowball: ")
    print(stemmeds)
    
    
def stemming_original(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
    
    # Obtenemos los tokens del texto.
    tokens = nltk.word_tokenize(text)
        
    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    return stemmeds


possible_lan = {"english":"en", "spanish":"es"}
def get_stemmed_txts(raw_texts):
    stemmed_txts = []
    for text in raw_texts:
        language = get_language(possible_lan, text)
        stemmed_txt = stemming_original(text, language)
        text = nltk.Text(stemmed_txt)
        stemmed_txts.append(text)
    return stemmed_txts
    
# stemming(text)

*EXPLORAR CONTENIDO DE LAS PÁGINAS WEB*

Ideas del propio enunciado de la práctica (muchas incluidas ya antes pero simmplemente para comprobar):

* Eliminar palabras vacías
* Stemming
* Lematización
* Reconocer entidades nombradas
* n-gramas
* entity linking (buscando información de los individus en recursos externos como Wikipedia u otros)


## Inicio programa

In [268]:
# Folder with all texts
folder = "Thomas_Baker"
# Empty list to hold text documents.
raw_texts = []
raw_texts_2 = []
named_ent_txts_1 = []
types_included_1 = ['NOUN']

possible_lan = {"english":"en", "spanish":"es"}

listing = os.listdir(folder)
for file in listing:
    if file.endswith(".txt"):
        url = folder+"/"+file
        f = open(url,encoding="latin-1");
        raw = f.read()
        f.close()
        raw_texts_2.append(raw)
        tokens = nltk.word_tokenize(raw)
        text = nltk.Text(tokens)
        raw_texts.append(text)

print("Prepared ", len(raw_texts), " documents...")
print("They can be accessed using texts[0] - texts[" + str(len(raw_texts)-1) + "]")

named_ent_txts_1 = get_named_ent_txts_1(raw_texts)
stemmed_txts = get_stemmed_txts(raw_texts_2)
print("lenght stemmed txts ", len(stemmed_txts))


# for text in raw_texts:
#     print("Length ", text[0:10])

# Similarity distance
distanceFunction ="cosine"
# distanceFunction = "euclidean"


# iterations for the evaluation of clusters
top_test = 10
priori_groups = 1

# Final solution for comparing in tests
# reference = [0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]
#groups = eval_test(top_test, priori_groups, reference)


tested_models = {}
tested_models["primitive"] = cluster_texts(raw_texts,4,distanceFunction)
tested_models["identity_analysis_1"] = cluster_texts(named_ent_txts_1,4,distanceFunction)
tested_models["stemmed_txts"] = cluster_texts(stemmed_txts,4,distanceFunction)


# test = cluster_texts(raw_texts,4,distanceFunction)
# print("test: ", test)
# Gold Standard
reference =[0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]
print("reference: ", reference)

# Evaluation

for model in tested_models:
    print("Model ", model, "; rand_score = ", adjusted_rand_score(reference,tested_models[model]))

#print("rand_score: ", adjusted_rand_score(reference,test))


Prepared  19  documents...
They can be accessed using texts[0] - texts[18]
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
steaming text...
lenght stemmed txts  19
reference:  [0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]
Model  identity_analysis_1 ; rand_score =  0.14027149321266968
Model  primitive ; rand_score =  0.17176317501626542
Model  stemmed_txts ; rand_score =  0.17176317501626542


In [269]:
# En la memoria se ha de justificar cada cambio propuesto.

# Se ha de comentar los resultados obtenidos.

# Siempre buscando una mejora en la clusterización